In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
import os
import sys

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [37]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score
import math
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Importing the dataset
Features = pd.read_csv('/content/drive/MyDrive/DataMining/Datasets/Selected_Features.csv')
X = Features.iloc[: ,:-1].values
Y = Features['class'].values


In [38]:
# # As this is a multiclass classification problem onehotencoding our Y.
# encoder = OneHotEncoder()
# Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [39]:
# splitting data
# smote_on_1 = 212
# smote = SMOTE(sampling_strategy={'Minimal': smote_on_1,'Moderate': smote_on_1, 'Extreme': smote_on_1})
# X, Y = smote.fit_resample(X, Y)
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((503, 26), (503, 4), (168, 26), (168, 4))

In [40]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((503, 26), (503, 4), (168, 26), (168, 4))

In [41]:
# making our data compatible to model.
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((503, 26, 1), (503, 4), (168, 26, 1), (168, 4))

In [42]:
# # # making our data compatible to model.
# # x_train = np.expand_dims(x_train, axis=2)
# # x_test = np.expand_dims(x_test, axis=2)
# y_train = np.expand_dims(y_train, axis=1)
# x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [43]:
import tensorflow as tf


In [44]:
model=Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
# model.add(Dropout(0.3))

model.add(Dense(units=4, activation='softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 26, 256)           1536      
                                                                 
 dropout_8 (Dropout)         (None, 26, 256)           0         
                                                                 
 conv1d_9 (Conv1D)           (None, 26, 256)           327936    
                                                                 
 dropout_9 (Dropout)         (None, 26, 256)           0         
                                                                 
 conv1d_10 (Conv1D)          (None, 26, 128)           163968    
                                                                 
 dropout_10 (Dropout)        (None, 26, 128)           0         
                                                                 
 conv1d_11 (Conv1D)          (None, 26, 64)           

In [45]:
# rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
# history=model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test), callbacks=[rlrp])
history=model.fit(x_train, y_train, batch_size=8, epochs=200, validation_data=(x_test, y_test))

Epoch 1/200
63/63 [==============================] - 4s 44ms/step - loss: 1.2557 - accuracy: 0.4354 - val_loss: 1.2105 - val_accuracy: 0.4286
Epoch 2/200
63/63 [==============================] - 3s 44ms/step - loss: 1.1273 - accuracy: 0.4911 - val_loss: 1.1108 - val_accuracy: 0.4702
Epoch 3/200
63/63 [==============================] - 3s 55ms/step - loss: 1.0398 - accuracy: 0.5467 - val_loss: 1.0879 - val_accuracy: 0.4702
Epoch 4/200
63/63 [==============================] - 3s 42ms/step - loss: 1.0347 - accuracy: 0.5547 - val_loss: 1.0725 - val_accuracy: 0.5000
Epoch 5/200
63/63 [==============================] - 4s 60ms/step - loss: 0.9935 - accuracy: 0.5626 - val_loss: 1.0464 - val_accuracy: 0.5000
Epoch 6/200
63/63 [==============================] - 2s 37ms/step - loss: 0.9746 - accuracy: 0.5706 - val_loss: 1.1059 - val_accuracy: 0.4821
Epoch 7/200
63/63 [==============================] - 2s 35ms/step - loss: 0.9490 - accuracy: 0.6103 - val_loss: 1.0654 - val_accuracy: 0.4881
Epoch 

In [46]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


In [47]:
yhat_probs = model.predict(x_test, verbose=0)
yhat_probs

array([[1.42155027e-12, 9.49854851e-01, 5.01384661e-02, 6.72440410e-06],
       [7.08313882e-02, 1.03330931e-04, 5.17053631e-05, 9.29013550e-01],
       [9.99997973e-01, 2.04744288e-06, 6.96336544e-25, 1.83006838e-14],
       [3.27004107e-27, 1.70588912e-03, 9.92806554e-01, 5.48744854e-03],
       [9.98589575e-01, 1.41036790e-03, 7.29901589e-15, 4.68269796e-08],
       [2.40593363e-05, 9.88000333e-01, 3.08340020e-08, 1.19756283e-02],
       [4.32841212e-08, 9.93619978e-01, 6.37015654e-03, 9.81711946e-06],
       [2.25476593e-29, 9.99656916e-01, 3.42987216e-04, 7.85748213e-08],
       [2.96352525e-19, 4.12145168e-01, 5.87854862e-01, 3.04478789e-08],
       [3.50582174e-26, 2.00560025e-04, 9.99799430e-01, 2.99388470e-09],
       [6.68311850e-06, 9.99979138e-01, 2.52773386e-10, 1.42330155e-05],
       [1.00000000e+00, 2.34020403e-09, 5.37798650e-26, 3.98004763e-12],
       [5.17536307e-07, 9.47214246e-01, 4.76231603e-14, 5.27851842e-02],
       [9.99407649e-01, 1.25312710e-07, 4.56027056e